In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import random

class ImageClassificationDataset(Dataset):
    def __init__(self, positive_dir, negative_dir, transform=None):
        self.positive_dir = positive_dir
        self.negative_dir = negative_dir
        self.transform = transform
        self.positive_files = [(os.path.join(positive_dir, f), 1) for f in os.listdir(positive_dir) if f.endswith('.png')]
        self.negative_files = [(os.path.join(negative_dir, f), 0) for f in os.listdir(negative_dir) if f.endswith('.png')]
        self.all_files = self.positive_files + self.negative_files

    def __len__(self):
        return len(self.all_files)

    def __getitem__(self, idx):
        file_path, label = self.all_files[idx]
        image = Image.open(file_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label, os.path.basename(file_path)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

positive_dir = "data/train_images/Train/Positive"
negative_dir = "data/train_images/Train/Negative"

# use an equal number of positive and negative samples for training
positive_files = [f for f in os.listdir(positive_dir) if f.endswith('.png')]
negative_files = [f for f in os.listdir(negative_dir) if f.endswith('.png')]
n = min(len(positive_files), len(negative_files))
positive_files = positive_files[:n]
negative_files = negative_files[:n]


# suffle split train and test data 85/15
random.shuffle(positive_files)
random.shuffle(negative_files)
split = int(0.85 * n)
train_positive_files = positive_files[:split]
train_negative_files = negative_files[:split]
test_positive_files = positive_files[split:]

train_dataset = ImageClassificationDataset(positive_dir, negative_dir, transform)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)

test_dataset = ImageClassificationDataset(positive_dir, negative_dir, transform)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=True)

In [2]:
import timm
import torch.nn as nn

class VisionTransformer(nn.Module):
    def __init__(self, num_classes=2):
        super(VisionTransformer, self).__init__()
        self.model = timm.create_model('vit_base_patch16_224', pretrained=False)
        self.model.head = nn.Linear(self.model.head.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

model = VisionTransformer(num_classes=2)

In [ ]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.01)  # Start with a higher learning rate

# Define the learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)  # Decrease LR by a factor of 0.9 every epoch

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels, _ in train_dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Step the learning rate scheduler
    scheduler.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_dataloader):.4f}, Learning Rate: {scheduler.get_last_lr()[0]:.6f}")

print("Training complete.")

In [ ]:
# import torch.optim as optim

# # Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Training loop
# num_epochs = 5
# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     for images, labels, _ in train_dataloader:
#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
    
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_dataloader):.4f}")

# print("Training complete.")

In [ ]:
# Function to evaluate the model and output file names
def evaluate_model(model, dataloader):
    model.eval()
    all_predictions = []
    all_labels = []
    all_filenames = []

    with torch.no_grad():
        for images, labels, filenames in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.numpy())
            all_labels.extend(labels.numpy())
            all_filenames.extend(filenames)

    return all_predictions, all_labels, all_filenames

predictions, labels, filenames = evaluate_model(model, test_dataloader)

# Print the results
for filename, prediction, label in zip(filenames, predictions, labels):
    print(f"File: {filename}, Prediction: {prediction}, Label: {label}")

In [ ]:
# percentag of correct predictions
correct = sum([1 if p == l else 0 for p, l in zip(predictions, labels)])
accuracy = correct / len(labels)
print(f"Accuracy: {accuracy:.2f}")
